# Análise de vinhos 🍷

## Sobre o dataset
O dataset é composto por informações sobre características químicas de diferentes tipos de vinho, branco ou tinto.
- Type: indica se o vinho é branco ou tinto;
- Fixed acidity: a quantidade de ácidos não volatéis (g/L);
- Volatile acidity: a quantidade de ácidos volatéis (g/L);
- Citric acid: a quantidade de ácido cítrico (g/L);
- Residual sugar: a quantidade de açúcar residual (g/L);
- Chlorides: a quantidade de cloretos (g/L);
- Free sulfur dioxide: a quantidade de dióxido de enxofre livre (mg/L);
- Total sulfur dioxide: a quantidade total de dióxido de enxofre (mg/L);
- Density: a densidade do vinho (g/cm³);
- pH: o nível de pH do vinho;
- Sulphates: a quantidade de sulfatos (g/L);
- Alcohol: o teor alcoólico do vinho (%vol);
- Quality: a avaliação do vinho (0-10)

# ----------------------------------------------------------------------------------

# Análise dos dados contidos no dataset

In [4]:
# importação das bibliotecas principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
# importação do dataset a ser utilizado
df = pd.read_csv('../dataset/VinhoClassReg.csv')

In [6]:
# visualização das primeiras linhas da tabela para compreender melhor a estrutura do dataset
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [8]:
# pesquisa de valores nulos no dataset que possam interferir na análise dos dados 
df.isnull().sum()

type                     0
fixed acidity           10
volatile acidity         8
citric acid              3
residual sugar           2
chlorides                2
free sulfur dioxide      0
total sulfur dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

### Interpretação dos resultados obtidos: 
- existem valores nulos nas colunas fixed acidity (10), volatile acidity (8), citric acid (3), residual sugar (2) e chlorides (2)
- uma vez que o número de valores nulos é resíduo em todas as colunas em que foram encontrados nulos, o próximo passo consiste em eliminar as linhas com valores nulos.

In [11]:
# remoção de linhas com valores nulos
df.dropna(inplace=True)

In [12]:
# nova pesquisa de valores nulos para confirmar que o passo anterior ocorreu com sucesso
df.isnull().sum()

type                    0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64